Imports

In [1]:
import multiprocessing as mp
import random
import pygad
from models import Order, SimulationEnvironment

Classes #TODO: outsource to py file and import into this one

In [3]:
class Agent(): # agent base class, setup for multiprocessing
    def __init__(self, result_size : int):
        # write results of each agent to this variable
        self.RESULT = mp.Array('f', result_size, True) #Note: f - indicates use of float values, True = Lock Object for synchronization is created
        self.CURRENT_STATE = mp.Array('f', result_size, True)
        self.CONTINUE = False
        
    def step(self, gen):
        self.CONTINUE = True
        
    def configure(self):
        pass

    def write_result(self, result):
        with self.RESULT.get_lock():
            self.RESULT = result

    def run():
        pass

class GreedyAgent(Agent):

    def confiugure(self, environment : SimulationEnvironment):
        self.environment = environment

    def pick_random(self, task, tasks):
        task = self.environment.get_task(task)
        if len(task.follow_up_tasks) > 0:
            follow_up_id = random.choice(task.follow_up_tasks)
            follow_up = self.environment.get_task(follow_up_id)
            tasks.append(follow_up)
            self.pick_random(follow_up.id, tasks)
        
    def pick_random_path(self, recipe):
        tasks = []
        task = random.choice(recipe.tasks)
        tasks.append(task)
        self.pick_random(task.id, tasks)
        return tasks
    
    def assignments_for(self, id):
        assignments = []
        for i in range(0, len(self.RESULT), 4):
            if self.RESULT[i] == id:
                assignments.push_back(self.RESULT[i], self.RESULT[i+1], self.RESULT[i+2], self.RESULT[i+3]) #TODO: double check encoding, should probably change encoding for end result
        return assignments
    
    def is_blocked(self, workstation_id, start_time, duration):
        assignments = self.assignments_for(workstation_id)
        if assignments:
            for assignment in assignments:
                assignment_start_time = assignment[1]
                assignment_duration = self.environment.get_duration(assignment[0], workstation_id)
                if start_time > assignment_start_time and start_time < assignment_start_time + assignment_duration:
                    return True
                if start_time + duration > assignment_start_time and start_time + duration < assignment_start_time + assignment_duration:
                    return True
                # if new task is longer than already scheduled task, it could start before and end after -> overlap
                if assignment_start_time > start_time and assignment_start_time < start_time + duration:
                    return True
                if assignment_start_time + assignment_duration > start_time and assignment_start_time + assignment_duration < start_time + duration:
                    return True
        return False

    def run(self, order : Order):
        recipes = []
        for resource in order.resources:
            recipes.append((order.id, resource.recipes))
        tasks = []
        for recipe in recipes:
            path = recipe[1][0].tasks # ignore possibility of multiple recipes for now
            path.reverse()
            tasks.append(path) # add in reverse order for each recipe
        success_count = 0
        for i in range(len(tasks)):
            prev_duration = 0
            for j in range(len(tasks[i])):
                task = tasks[i][j]
                workstations = self.environment.get_valid_workstations(task.external_id)
                chosen_workstation = None
                for workstation in workstations:
                    duration = self.environment.get_duration(task.external_id, workstation.external_id)
                    start_slot = order.delivery_time - prev_duration - duration
                    if not self.is_blocked(workstation.external_id, start_slot, duration):
                        chosen_workstation = workstation
                        break            
                if chosen_workstation:
                    prev_duration += duration
                    self.RESULT.push_back(chosen_workstation)
                    self.RESULT.push_back(start_slot)
                    self.RESULT.push_back(task.external_id)
                    self.RESULT.push_back(order.external_id)
                    success_count += 1
                else:
                    if not order.divisible: # if order can not only be partially fullfilled too
                        return False
                    else:
                        break
        return success_count > 0

class GAAgent(Agent): # no compression stage

    def configure(self, environment : SimulationEnvironment, earliest_time_slot, last_time_slot):
        self.environment = environment
        self.last_time_slot = last_time_slot
        self.earliest_time_slot = earliest_time_slot
        GAAgent.last_time_slot = last_time_slot # should be the same for all agents anyway
        GAAgent.earliest_time_slot = earliest_time_slot

    def is_feasible(solution):
        for i in range(len(solution)):
            start = solution[i]
            end = start + duration_lookup_table[operations[i]][assignments[i]] # TODO: format input data
            if end > GAAgent.last_time_slot:
                return False
            if start < GAAgent.earliest_time_slot:
                return False
            # overlap
            same_workstation = [] # append indices of jobs running on the same workstation
            for j in range(len(assignments)):
                if assignments[j] == assignments[i] and i != j:
                    same_workstation.append(j)
            for j in range(len(same_workstation)):
                other_start = solution[same_workstation[j]]
                other_end = other_start + duration_lookup_table[operations[same_workstation[j]]][assignments[same_workstation[j]]]
                if start > other_start and start < other_end:
                    return False
                if end > other_start and end < other_end:
                    return False
                if other_start > start and other_start < end:
                    return False
                if other_end > start and other_end < end:
                    return False
            # check sequence
            order = order_for_index[i]
            if i > 0:
                if order_for_index[i-1] == order: # same order
                    prev_start = solution[i-1]
                    prev_end = prev_start + duration_lookup_table[operations[i-1]][assignments[i-1]]
                    if prev_end > start:
                        return False
        return True

    def mutation_function_time_slots(offsprings, ga_instance):
        for offspring in offsprings:
            p = 1 / len(offspring)
            for i in range(len(offspring)):
                if random.random() < p:
                    offspring[i] = random.randint(GAAgent.earliest_time_slot, GAAgent.last_time_slot)
        return offsprings

    def fitness_function_time_slots(solution, solution_idx):
        fitness = 1
        if not GAAgent.is_feasible(solution):
            #fitness += last_time_slot
            return -2 * GAAgent.last_time_slot
        max = -float('inf')
        min = float('inf')
        for i in range(len(solution)):
            start = solution[i]
            end = start + duration_lookup_table[operations[i]][assignments[i]]
            if start < min:
                min = start
            if end > max:
                max = end
        fitness += abs(max - min)
        return -fitness

    def run(self, order : Order = None):
        time_best = []
        average_time = []
        assignments = solution # TODO: set the solution input -> formatting of the result
        start_slots = []
        for assignment in assignments:
            start_slots.append(0)
        num_genes = len(start_slots)
        num_generations = 5000
        num_parents_mating = 50
        sol_per_pop = 100
        init_range_low = 0
        init_range_high = self.last_time_slot
        parent_selection_type = 'rws'
        keep_parents = 10
        crossover_type = 'two_points'
        mutation_type = GAAgent.mutation_function_time_slots
        #mutation_type = 'random'
        mutation_percentage_genes = 10
        fitness_func = GAAgent.fitness_function_time_slots
        gene_type = int
        space_time = {'low': self.earliest_time_slot, 'high': self.last_time_slot}
        gene_space = []
        for i in range(len(assignments)):
            gene_space.append(space_time)
        ga_instance = pygad.GA(num_generations=num_generations, num_parents_mating=num_parents_mating, fitness_func=fitness_func, sol_per_pop=sol_per_pop, num_genes=num_genes, init_range_low=init_range_low, init_range_high=init_range_high, parent_selection_type=parent_selection_type, keep_parents=keep_parents, crossover_type=crossover_type, mutation_type=mutation_type, mutation_percent_genes=mutation_percentage_genes, gene_type=gene_type, gene_space=gene_space)
        ga_instance.run()
        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        print("Parameters of the best solution : {solution}".format(solution=solution))
        print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(solution_fitness) - 1))
        

Helper Functions

In [ ]:
def create_agent(type, result_size): # TODO: add problem information
    if type == 0: # greedy
        return GreedyAgent(result_size)
    return None

Setups

In [ ]:
agents = []
result_size = 10 # amount of tasks, exact size differs by agent
# for homogenous systems
n = 10
types = []
for i in range(n):
    types.append(0)

# for heterogenous systems
types = [0, 1, 1, 2, 0, 3, 3, 4, 1, 0] # example


for type in types:
    agents.append(create_agent(type, result_size)) # add n agents

Run

In [ ]:
results = []
processes = []
for agent in agents:
    process = mp.Process(target=agent.run(), args=())
    process.start()
    processes.append(process)
    
for i in range(len(processes)):
    processes[i].join()
    results.append(agents[i].RESULT)


In [ ]:
results = []
processes = []

for agent in agents:
    process = mp.Process(target=agent.run(), args=())
    process.start()
    processes.append(process)

max_gen = 100
for i in range(max_gen):
    for agent in agents:
        agent.step(i)
    states = []
    for agent in agents:
        states.append(agent.CURRENT_STATE) # TODO: common format for current_state in all different agent types needed
        # compare states, etc.

for i in range(len(processes)):
    processes[i].join()
    results.append(agents[i].RESULT)
